In [6]:
import pandas as pd

In [7]:
file_path = []
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp02.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp04.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp05.csv')
file_path.append(r'C:\Users\ADMIN\Desktop\TASK_CLEAN_DATA\transactions\transactions_pp06.csv')

In [24]:
# print(file_path[0])
df_pp02 = pd.read_csv(file_path[0])
df_pp02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3293 entries, 0 to 3292
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   transaction_id               3293 non-null   object
 1   transaction_event_code       3293 non-null   object
 2   transaction_initiation_date  3293 non-null   object
 3   transaction_updated_date     3293 non-null   object
 4   transaction_amount           3293 non-null   object
 5   transaction_status           3293 non-null   object
 6   transaction_subject          88 non-null     object
 7   ending_balance               3293 non-null   object
 8   available_balance            3293 non-null   object
 9   protection_eligibility       3293 non-null   int64 
 10  payer_info                   3293 non-null   object
 11  shipping_info                3293 non-null   object
 12  cart_info                    3293 non-null   object
 13  store_info                   3293

In [ ]:
transactions = {}
for col in df_pp02:
    for i in range(19)
    print(df_pp02[col].iloc[i])
    transactions[col] = df_pp02[col].iloc[0]
    break

print(transactions)


78W97559FU6028206
{'transaction_id': '78W97559FU6028206'}


In [26]:
import yaml
import pyodbc
import urllib.parse
from sqlalchemy import create_engine

server = "DESKTOP-2PSQA2P\\MSSQL_SERVER"
database = "TASK"
username = "DESKTOP-2PSQA2P\\ADMIN"

connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    Trusted_Connection=yes;
"""
conn = pyodbc.connect(connection_string)
print("Connection succesful")

params = urllib.parse.quote_plus(connection_string)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


Connection succesful


In [27]:
table_name = 'transactions_pp02'
df_pp02.to_sql(table_name, con=engine, if_exists='replace', index=False)

103